In [1]:
import numpy as np

In [2]:
def parse_calibration_file(file_path):
    """
    Parse the calibration file and load the calibration matrices into numpy arrays.

    Args:
        file_path (str): The path to the calibration text file.

    Returns:
        dict: A dictionary containing the calibration matrices.
    """
    calib_data = {}
    
    with open(file_path, 'r') as file:
        lines = file.readlines()
    
    for line in lines:
        line = line.strip()
        if not line or line.startswith('calib_time') or line.startswith('corner_dist'):
            continue
        
        key, value = line.split(':', 1)
        key = key.strip()
        values = np.fromstring(value, sep=' ')
        
        if 'S_' in key or 'S_rect_' in key:
            calib_data[key] = values.reshape((2,))
        elif 'K_' in key or 'R_' in key or 'R_rect_' in key:
            calib_data[key] = values.reshape((3, 3))
        elif 'D_' in key:
            calib_data[key] = values
        elif 'T_' in key:
            calib_data[key] = values.reshape((3,))
        elif 'P_rect_' in key:
            calib_data[key] = values.reshape((3, 4))
        else:
            raise ValueError(f"Unknown calibration key: {key}")
    
    return calib_data

In [3]:
calib_file_path = '/home/DSGN_DockerizeAndDeploy/data/DrivingStereoSet/training/calib/000587.txt'
calib_data = parse_calibration_file(calib_file_path)
# for key, matrix in calib_data.items():
#     print(f"{key}: \n{matrix}\n")

In [12]:
# K= calib_data['K_101']
# T = np.hstack((calib_data['R_101'], calib_data['T_101'].reshape(3, 1)))
# img_shape = calib_data['S_101']
# img_shape_rect = calib_data['S_rect_101']
# width_rect_ratio = img_shape_rect[0] / img_shape[0]
# height_rect_ratio = img_shape_rect[1] / img_shape[1]
# K_rect = K.copy()
# K_rect[0, 0] *= width_rect_ratio
# K_rect[0, 2] *= width_rect_ratio
# K_rect[1, 1] *= height_rect_ratio
# K_rect[1, 2] *= height_rect_ratio

# print(f'img shape: {img_shape}')
# print(f'img shape rect: {img_shape_rect}')
# print(f'T: {T.shape}: \n{T}\n')
# print(f'K: {K.shape}: \n{K}\n')
# print(f'K_rect: {K_rect.shape}: \n{K_rect}\n')
# print(f'Ground Truth: P2_rect {calib_data["P_rect_101"].shape}: \n{calib_data["P_rect_101"]}\n')

In [20]:
P2_rect = calib_data['P_rect_101']
P2_test =  P2_rect @ np.linalg.inv(calib_data['R_rect_101'])

print(f'P2_test: {P2_test.shape}: \n{P2_test}\n')
print(f'P2_rect: {P2_rect.shape}: \n{P2_rect}\n')

ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 3 is different from 4)

In [16]:
K= calib_data['K_101']
T = np.hstack((calib_data['R_101'], calib_data['T_101'].reshape(3, 1)))
P2_test = K @ T

print(f'K: {K.shape}: \n{K}\n')
print(f'T: {T.shape}: \n{T}\n')
print(f'P2_test: {P2_test.shape}: \n{P2_test}\n')

K: (3, 3): 
[[ 2.0695e+03 -2.0000e-01  9.7410e+02]
 [ 0.0000e+00  2.0683e+03  5.9000e+02]
 [ 0.0000e+00  0.0000e+00  1.0000e+00]]

T: (3, 4): 
[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]]

P2_test: (3, 4): 
[[ 2.0695e+03 -2.0000e-01  9.7410e+02  0.0000e+00]
 [ 0.0000e+00  2.0683e+03  5.9000e+02  0.0000e+00]
 [ 0.0000e+00  0.0000e+00  1.0000e+00  0.0000e+00]]

